In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import pandas as pd
spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.read.csv('A_lvr_land_A.csv', header=True, inferSchema=True)
df = df.union(spark.read.csv('B_lvr_land_A.csv', header=True, inferSchema=True)) 
df = df.union(spark.read.csv('E_lvr_land_A.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('F_lvr_land_A.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('H_lvr_land_A.csv', header=True, inferSchema=True))
#篩選資料
df = df.filter(df['主要用途'].contains('住家用')).filter(df['建物型態'].contains('住宅大樓')).filter(~df['總樓層數'].isin(['一層','二層','三層','四層','五層','六層','七層','八層','九層','十層','十一層','十二層']))
df = df.sort(desc('交易年月日'))

In [ ]:
print(df.count())

In [31]:
# df.printSchema()
# df.dtypes
df.createOrReplaceTempView('df')
spark.sql("select * from df ").show(10)



+--------+--------------------+----------------------------------------+----------------------+----------------+------------------+------------------+----------+---------------+--------+--------+--------------------------+--------+--------------+------------+----------------------+---------------+---------------+---------------+-----------------+------------+---------+--------------+--------+------------------------+----------+------------------------------------+-------------------+----------+------------+--------+----+--------+
|鄉鎮市區|            交易標的|                        土地位置建物門牌|土地移轉總面積平方公尺|都市土地使用分區|非都市土地使用分區|非都市土地使用編定|交易年月日|     交易筆棟數|移轉層次|總樓層數|                  建物型態|主要用途|      主要建材|建築完成年月|建物移轉總面積平方公尺|建物現況格局-房|建物現況格局-廳|建物現況格局-衛|建物現況格局-隔間|有無管理組織|   總價元|單價元平方公尺|車位類別|車位移轉總面積(平方公尺)|車位總價元|                                備註|               編號|主建物面積|附屬建物面積|陽台面積|電梯|移轉編號|
+--------+--------------------+----------------------------------------+----------------------+----------------+------